In [1]:
%load_ext autoreload
%autoreload 2

import os 
# os.environ['R_HOME']= r'C:\Users\tomha\miniconda3\envs\octagon_analysis\lib\R'
os.environ['R_HOME']= r'D:\Users\Tom\miniconda3\envs\octagon_analysis\lib\R'

import rpy2.robjects as robjects
print(robjects.r('R.version.string'))

import parse_data.prepare_data as prepare_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import globals
import data_strings
import data_extraction.get_indices as get_indices
import analysis.wall_visibility_and_choice as wall_visibility_and_choice
from trajectory_analysis import trajectory_vectors
from plotting import plot_octagon
import parse_data.identify_filepaths as identify_filepaths 
from data_extraction.trial_list_filters import filter_trials_other_visible
from analysis import opponent_visibility


[1] "R version 4.1.3 (2022-03-10)"



### Create a dataframe to feed into a GLM using D2H, D2L, First Seen, Wall Separation, and PlayerID (random effect) to predict P(Choose High)

In [2]:
data_folder = data_strings.DATA_FOLDER
json_filenames_social, json_filenames_solo = identify_filepaths.get_filenames()

In [3]:
# # restrict data for testing
# json_filenames_social = json_filenames_social[:8]
# json_filenames_solo = json_filenames_solo[:32]

In [4]:
df, trial_lists_social = prepare_data.prepare_data(data_folder, json_filenames_social, combine=False)

filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\pseudonymised_json_files\240913_1\2024-09-13_11-31-00_YW13_JL13_Social.json
Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\pseudonymised_json_files\240927_1\2024-09-27_14-25-20_SH27_EN27_Social.json
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\pseudonymised_json_files\241017_1\2024-10-17_14-28-40_SP17_AW17_Social.json
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\pseudonymised_json_files\241017_2\2024-10-17_16-41-38_ZH17_EM17_Social.json
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\pseudonymised_json_files\241112_1\2024-11-12_13-31-14_KA12_WM12_Social.json
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\pseudonymised_json_files\2411

In [5]:
df, trial_lists_solo = prepare_data.prepare_data(data_folder, json_filenames_solo, combine=False)

filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\pseudonymised_json_files\240913_1\2024-09-13_11-23-37_YW13_FirstSolo.json
Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\pseudonymised_json_files\240913_1\2024-09-13_11-53-34_YW13_SecondSolo.json
Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\pseudonymised_json_files\240913_1\2024-09-13_11-23-50_JL13_FirstSolo.json
Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\pseudonymised_json_files\240913_1\2024-09-13_11-53-56_JL13_SecondSolo.json
Data is from period before 2024-09-13 00:00:00
Running datafram

In [6]:
# create a list of combined pre- and post- social solo sessions, removing 5 trials from each pre
trial_lists_combined_solo = []
cut_trials = 5
for i in range(0,len(trial_lists_solo), 2): # iterate over each individual
    # get the trial lists for both solo sessions
    trial_list_first_solo = trial_lists_solo[i]
    trial_list_second_solo = trial_lists_solo[i + 1]

    # cut first cut_trials trials (learning controls/associations) from the first solo
    trial_list_first_solo = trial_list_first_solo[cut_trials:]

    # combine trial lists from the first and second solo sessions (the current and consecutive index)
    trial_list = trial_list_first_solo + trial_list_second_solo

    trial_lists_combined_solo.append(trial_list)


In [7]:
len(trial_lists_combined_solo), len(trial_lists_social)

(64, 32)

### Regressor value extraction functions (for one session)

In [8]:
def extract_wall_sep(trial_list):
    ''' Return wall separation for one session.
        1 for 45 degrees, 2 for 90 and 4 for 180. '''
    
    wall_sep = np.full(len(trial_list), np.nan)
    for i, trial in enumerate(trial_list):
        wall_sep_this_trial = get_indices.get_wall_difference(trial=trial)
        wall_sep[i] = wall_sep_this_trial

    return wall_sep


def extract_first_wall_seen(trial_list, player_id):
    ''' Return first visible walls for one player across one session.
        2 for WALL_1, 1 for WALL_2, np.nan for no visible wall (or both initially visible).
        Note opposite values to wall indices '''
        
    high_wall_first_visible_session = wall_visibility_and_choice.get_given_wall_first_visible_session(trial_list,
                                                                                                        player_id,
                                                                                                        wall_index=0,
                                                                                                        current_fov=110)

    low_wall_first_visible_session = wall_visibility_and_choice.get_given_wall_first_visible_session(trial_list,
                                                                                                        player_id, 
                                                                                                        wall_index=1,
                                                                                                        current_fov=110)
    high_wall_first_visible_session = high_wall_first_visible_session*2
    
    first_visible_session = high_wall_first_visible_session + low_wall_first_visible_session

    first_visible_session[first_visible_session == 0] = np.nan

    return first_visible_session

# double check code
def extract_distances_to_walls(trial_list, player_id, normalise=False):
    ''' Return a trial_num, 2 sized array, where column 1
        is distance to WALL_1, and column 2 is distance to WALL_2.
        Data applies to one full session, and specified player_id.
        If normalise, returns distances as a proportion of the maximum
        possible in the arena '''
    
    # get octagon alcove coordinates
    alcove_coordinates = plot_octagon.return_alcove_centre_points()

    positions_session = np.full((len(trial_list), 2), np.nan)
    walls_session = np.full((len(trial_list), 2), np.nan)
    distances_session = np.full((len(trial_list), 2), np.nan)

    # get distances for each trial in the session
    for i, trial in enumerate(trial_list):
        # get WALL_1 and WALL_2 coordinates
        trial_walls = get_indices.get_walls(trial)
        high_wall_idx = trial_walls[0] - 1
        low_wall_idx = trial_walls[1] - 1
        trial_high_coordinates = alcove_coordinates[:,high_wall_idx]
        trial_low_coordinates = alcove_coordinates[:, low_wall_idx]

        # index trajectory at timepoint 0 to get player starting coordinates
        trajectory = trajectory_vectors.extract_trial_player_trajectory(trial=trial, player_id=player_id)
        trial_start_position = trajectory[:,0]

        # find distance between start position and WALL_1/WALL_2
        d2h = np.linalg.norm(trial_high_coordinates - trial_start_position) # WALL_1
        d2l = np.linalg.norm(trial_low_coordinates - trial_start_position) # WALL_2

        walls_session[i,:] = trial_walls 
        positions_session[i,:] = trial_start_position
        distances_session[i,:] = np.hstack((d2h, d2l))

    # normalise to maximum possible distance in octagon
    if normalise:
        distances_session = distances_session/plot_octagon.return_maximum_distance()

    return distances_session


def extract_opponent_visibility_slice_onset(trial_list, player_id, current_fov=110):
    ''' Return opponent visibility at slice onset for one player for one session.
        1 for opponent visible, 0 for opponent not visible '''
    
    # slice onset angle of Other from self centre FoV
    orientation_angle_to_other_session = opponent_visibility.get_angle_of_opponent_from_player_session(player_id, trial_list)

    # boolean array of Other visible
    other_visible_session = opponent_visibility.get_other_visible_session(orientation_angle_to_other_session, current_fov)
    other_visible_session = other_visible_session.astype(int) # converted to int for categorical regressor

    # does this return 1 and 0? 

    return other_visible_session


def extract_player_choice(trial_list, player_id, inferred_choice=True, debug=True):
    ''' Return (inferred by default) player choice for one player for one session.
        Where inferred and actual choice are both missing, values are np.nan '''

    # array of wall numbers where player choice is available, np.nan where it is not
    player_choice = wall_visibility_and_choice.get_player_wall_choice(trial_list, player_id,
                                                                        inferred_choice=inferred_choice, debug=debug)

    # 2 where player chose High, 0 where player chose Low, np.nan where lacking inferred choice
    high_wall_chosen_session = get_indices.was_given_wall_chosen(trial_list, player_choice,
                                                                    given_wall_index=0)
    high_wall_chosen_session = high_wall_chosen_session*2
    print(f"High wall chosen session:\n{high_wall_chosen_session}")

    # 1 where player chose Low, 0 where player chose High, np.nan where lacking inferred choice
    low_wall_chosen_session  = get_indices.was_given_wall_chosen(trial_list, player_choice,
                                                                    given_wall_index=1)
    
    print(f"Low wall chosen session:\n{low_wall_chosen_session}")

    # 1 or 2 where player chose Low or High respectively, np.nan where lacking inferred choice
    chosen_wall_session = high_wall_chosen_session + low_wall_chosen_session

    print(f"Overall chosen wall for this session:\n{chosen_wall_session}")

    # Does this switch to 0 or 1 respectively and np.nan? 
    chosen_wall_session = chosen_wall_session -1 

    return chosen_wall_session


def extract_trial_outcome(trial_list, player_id):
    ''' Return whether this player won the trial for one player for one session '''
    
    trigger_activators = get_indices.get_trigger_activators(trial_list)
    this_player_won_session = (trigger_activators-1)*-1 if player_id == 0 else trigger_activators

    return this_player_won_session

In [9]:
test = extract_player_choice(trial_lists_social[3], 0)

d:\tom\documents\swc\repos\octagon_analysis\trajectory_analysis\trajectory_vectors.py:321: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))


Time taken for infer_loser_choice_session (one session) is 1.18
Loser inferred choices: [6. 1. 2. 6. 8. 2. 7. 3. 6. 5. 1. 6. 1. 4. 8. 2. 7. 7. 4. 4. 2. 8. 3. 8.
 2. 5. 1. 4. 7. 6. 4. 8. 7. 2. 1. 3. 5. 1. 3. 5. 4. 3. 8. 2. 8. 1. 4. 6.
 8. 4. 4. 1. 7. 5. 7. 8. 2. 5. 3. 8. 5. 3. 7. 1. 8. 4. 1. 7. 6. 3. 3. 1.
 7. 1. 2. 1. 2. 1. 4. 2. 5. 5. 3. 7. 4. 2. 4. 6. 6. 3. 8. 7. 7. 1. 4. 6.
 5. 2. 5. 4. 4. 6. 4. 4. 4. 8. 2. 5. 8. 5. 1. 6. 1. 8. 5. 6. 1. 8. 8. 8.
 3. 6. 1. 5. 5. 2. 1. 2. 5. 3. 7. 7. 1. 8. 1. 8. 8. 8. 8. 8. 6. 1. 1. 6.
 2. 3. 7. 5. 1. 5. 8. 2. 7. 8. 6. 2. 4. 8. 8. 2. 2. 1. 3. 7. 2. 1. 6. 6.
 1. 7. 1. 3. 2. 5. 6. 3.]
proportion loser's choice confident = 0.5965909090909091
Time taken for player_wall_choice_win_or_loss (one session for one player) is 1.32
Chosen walls are [5. 1. 3. 8. 3. 2. 7. 3. 6. 8. 5. 6. 1. 7. 2. 2. 2. 7. 4. 3. 2. 8. 4. 8.
 2. 4. 2. 8. 6. 5. 4. 1. 8. 6. 4. 4. 8. 4. 3. 5. 4. 4. 6. 4. 8. 8. 5. 6.
 8. 4. 8. 5. 3. 6. 7. 4. 2. 6. 8. 4. 5. 7. 3. 3. 4. 3. 1. 8. 1. 3. 3. 8.

In [10]:
np.sum(~np.isnan(test))/test.size

np.float64(0.6534090909090909)

#### Filtering function

In [11]:
def filter_valid_trial_indices(trial_list, player_id, solo=False, return_filtered_data=True, return_original_indices=True):
    ''' Return the indices of a filtered trial list that correspond to trials in which there is a 
        retrievable choice and an identifiable first seen wall.'''


    original_indices = np.arange(len(trial_list))

    # identify indices of trial list with HighLow trials
    high_low_trial_indices = get_indices.get_trials_trialtype(trial_list, trial_type=globals.HIGH_LOW)

    print(f"{high_low_trial_indices.size} high_low_trial_indices for player_id {player_id}")


    # get choice and first visible wall data for one player, session
    if not solo:
        player_choice_session = extract_player_choice(trial_list, player_id)
    else:
        player_choice_session = extract_player_choice(trial_list, player_id, inferred_choice=False)
    
    first_wall_seen_session = extract_first_wall_seen(trial_list, player_id)

    # apply masks for one visible wall and a retrievable choice to the set of HighLow trials
    retrievable_choice_mask = ~np.isnan(player_choice_session[high_low_trial_indices])

    print(f"{np.sum(retrievable_choice_mask)} retrievable choices for player_id {player_id}")

    one_wall_first_visible_mask = ~np.isnan(first_wall_seen_session[high_low_trial_indices])

    print(f"{np.sum(one_wall_first_visible_mask)} first visible walls for player_id {player_id}")


    # combine masks
    final_mask = retrievable_choice_mask & one_wall_first_visible_mask

    # filter the trial list and analysis based on masks
    filtered_indices = high_low_trial_indices[final_mask]
    player_choice_filtered = player_choice_session[high_low_trial_indices][final_mask]
    one_wall_first_visible_filtered = first_wall_seen_session[high_low_trial_indices][final_mask]
    original_indices = original_indices[high_low_trial_indices][final_mask]


    assert filtered_indices.size > 0
    
    print(f"{filtered_indices.size} filtered trials for player_id {player_id}")
    
    if return_filtered_data and return_original_indices:
        return filtered_indices, player_choice_filtered, one_wall_first_visible_filtered, original_indices
    elif return_original_indices:
        return filtered_indices, original_indices
    elif return_filtered_data:
        return filtered_indices, player_choice_filtered, one_wall_first_visible_filtered
    else:
        return filtered_indices

### Create a dictionary to hold, for each session and player, regressor values for the session, only including trials with fully-populated regessors
- Fields for each of the regressors applied to all trials
- Fields for each of the regressors with only valid trials


In [12]:
solo = False
player_ids = [0] if solo else [0,1]


In [13]:
analysis_results = {
    experiment_id: {
        player_id: {
            session_type: {

                'regressors': {
                    'wall_sep': None,
                    'first_seen': None,
                    'd2h': None,
                    'd2l': None,
                    'opponent_visible': None,
                    'd2h_opponent': None,
                    'd2l_opponent': None
                },

                'dependent': {
                    'choice': None
                },

                'misc': {
                    'valid_trial_indices': None
                }
                
            }
            for session_type in ['solo', 'social']
        }   
        for player_id in player_ids
    }
    for experiment_id in np.arange(len(trial_lists_social))
}

### Populate the dictionary with data

In [15]:
a= analysis_results[4][1]['solo']['regressors']['first_seen'] 
_,_, b =filter_valid_trial_indices(trial_lists_combined_solo[9], 0, solo=True, return_original_indices=False)


51 high_low_trial_indices for player_id 0
Time taken for player_wall_choice_wins_only (one session for one player) is 0.04
include loser's inferred choice status: False
player wall choices for this player: [8. 3. 7. 7. 1. 1. 3. 2. 1. 2. 4. 1. 3. 7. 8. 3. 8. 3. 3. 6. 6. 5. 3. 1.
 8. 4. 3. 3. 2. 4. 2. 1. 1. 1. 8. 2. 5. 3. 1. 7. 7. 7. 5. 3. 6. 4. 8. 1.
 4. 6. 2. 2. 4. 8. 3. 1. 5. 3.]
High wall chosen session:
[0. 2. 0. 2. 2. 2. 2. 0. 2. 2. 0. 2. 2. 0. 2. 2. 0. 0. 0. 0. 0. 0. 2. 2.
 2. 2. 2. 2. 0. 2. 0. 0. 0. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2. 0. 2. 0. 0. 2.
 0. 2. 2. 2. 0. 2. 0. 2. 2. 2.]
Low wall chosen session:
[1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0.
 1. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
Overall chosen wall for this session:
[1. 2. 1. 2. 2. 2. 2. 1. 2. 2. 1. 2. 2. 1. 2. 2. 1. 1. 1. 1. 1. 1. 2. 2.
 2. 2. 2. 2. 1. 2. 1. 1. 1. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 1. 2. 1. 1. 2.
 1. 2. 2. 2. 1. 2. 1. 2. 

In [17]:
for experiment_id, players in analysis_results.items():
    for player_id, data in players.items():
        
        # get opponent_id
        opponent_id = 1 if player_id == 0 else 0
        
        # get the trial lists for this session and player
        trial_list_social = trial_lists_social[experiment_id]
        trial_list_solo = trial_lists_combined_solo[experiment_id*2 + player_id] # player_id used to select correct solo

        # filter the trial list for trials with valid data for all regressors
        print(f"Trial list social length for experimentId {experiment_id} and playerId {player_id}: {len(trial_list_social)}")
        (filtered_valid_trial_indices_social,
          filtered_retrievable_choice_social,
          filtered_first_wall_seen_social,
          original_indices_social) = filter_valid_trial_indices(trial_list_social, player_id)
        trial_list_social_filtered = [trial_list_social[i] for i in filtered_valid_trial_indices_social]
        
        (filtered_valid_trial_indices_solo,
          filtered_retrievable_choice_solo,
          filtered_first_wall_seen_solo,
          original_indices_solo) = filter_valid_trial_indices(trial_list_solo, player_id=0, solo=True)  # no inferred for solo
        trial_list_solo_filtered = [trial_list_solo[i] for i in filtered_valid_trial_indices_solo]
        
        ## social, use player_id == player_id and trial_list_social_filtered for functions
        # regressors social
        player_data = analysis_results[experiment_id][player_id]['social']
        distances = extract_distances_to_walls(trial_list_social_filtered, player_id)
        distances_opponent = extract_distances_to_walls(trial_list_social_filtered, player_id=opponent_id)
        player_data['regressors']['wall_sep'] = extract_wall_sep(trial_list_social_filtered)
        player_data['regressors']['first_seen'] = filtered_first_wall_seen_social
        player_data['regressors']['d2h'] = distances[:,0]
        player_data['regressors']['d2l'] = distances[:,1]
        player_data['regressors']['opponent_visible'] = extract_opponent_visibility_slice_onset(trial_list_social_filtered, player_id)
        player_data['regressors']['d2h_opponent'] = distances_opponent[:,0]
        player_data['regressors']['d2l_opponent'] = distances_opponent[:,1]


        # dependent variable social
        player_data['dependent']['choice'] = filtered_retrievable_choice_social

        # misc
        player_data['misc']['valid_trial_indices'] = original_indices_social


        ## solo, use player_id == 0 and trial_list_solo_filtered for functions
        # regressors solo
        player_data = analysis_results[experiment_id][player_id]['solo']
        distances = extract_distances_to_walls(trial_list_solo_filtered, player_id=0)
        player_data['regressors']['wall_sep'] = extract_wall_sep(trial_list_solo_filtered)
        player_data['regressors']['first_seen'] = filtered_first_wall_seen_solo
        player_data['regressors']['d2h'] = distances[:,0]
        player_data['regressors']['d2l'] = distances[:,1]

        # dependent variable solo
        player_data['dependent']['choice'] = filtered_retrievable_choice_solo # no inferred for solo

        # misc
        player_data['misc']['valid_trial_indices']






Trial list social length for experimentId 0 and playerId 0: 129
102 high_low_trial_indices for player_id 0
Time taken for infer_loser_choice_session (one session) is 0.82
Loser inferred choices: [2. 2. 8. 6. 1. 4. 3. 1. 1. 4. 7. 4. 2. 5. 6. 7. 1. 4. 2. 8. 6. 3. 1. 2.
 1. 1. 7. 3. 6. 2. 2. 5. 8. 5. 1. 5. 6. 5. 1. 3. 7. 1. 8. 5. 1. 7. 6. 4.
 2. 5. 7. 4. 4. 3. 7. 4. 4. 2. 7. 2. 1. 8. 5. 4. 3. 7. 3. 4. 1. 5. 1. 2.
 5. 6. 3. 2. 8. 6. 1. 1. 6. 2. 2. 7. 7. 3. 8. 4. 4. 1. 7. 1. 5. 4. 7. 3.
 8. 2. 1. 4. 8. 3. 3. 6. 5. 6. 3. 1. 2. 5. 5. 7. 1. 6. 5. 7. 3. 4. 5. 4.
 7. 4. 8. 5. 8. 4. 7. 3. 7.]
proportion loser's choice confident = 0.5968992248062015
Time taken for player_wall_choice_win_or_loss (one session for one player) is 0.92
Chosen walls are [2. 2. 5. 6. 4. 3. 5. 8. 8. 8. 3. 6. 4. 5. 4. 3. 1. 6. 3. 1. 2. 4. 3. 2.
 8. 1. 8. 5. 5. 2. 3. 5. 1. 4. 3. 6. 2. 8. 2. 7. 3. 2. 7. 4. 5. 8. 6. 4.
 2. 6. 2. 6. 1. 1. 7. 6. 6. 6. 7. 3. 4. 2. 5. 8. 4. 7. 5. 3. 3. 4. 1. 1.
 4. 2. 7. 1. 8. 2. 1. 1. 6. 2. 2. 3

d:\tom\documents\swc\repos\octagon_analysis\trajectory_analysis\trajectory_vectors.py:321: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))


Time taken for infer_loser_choice_session (one session) is 0.71
Loser inferred choices: [1. 1. 1. 1. 7. 1. 4. 1. 6. 5. 3. 6. 1. 3. 6. 3. 8. 4. 1. 8. 4. 5. 5. 4.
 1. 3. 1. 8. 7. 6. 1. 1. 1. 1. 1. 8. 1. 3. 7. 8. 7. 6. 8. 4. 7. 3. 5. 7.
 5. 1. 1. 4. 3. 1. 4. 4. 2. 2. 8. 1. 2. 8. 1. 3. 1. 1. 5. 7. 4. 1. 3. 7.
 5. 3. 6. 1. 6. 3. 3. 7. 1. 7. 1. 2. 6. 1. 2. 1. 1. 5. 7. 1. 7. 1. 1. 1.
 1. 5. 2. 1. 1. 8. 2. 5.]
proportion loser's choice confident = 0.4230769230769231
Time taken for player_wall_choice_win_or_loss (one session for one player) is 0.79
Chosen walls are [4. 5. 1. 2. 7. 7. 4. 2. 2. 5. 7. 1. 7. 7. 6. 7. 1. 2. 4. 6. 8. 3. 1. 4.
 8. 2. 3. 4. 4. 3. 4. 8. 1. 1. 8. 5. 5. 4. 5. 5. 7. 7. 1. 4. 7. 1. 8. 6.
 7. 8. 7. 8. 2. 7. 4. 4. 2. 2. 8. 1. 2. 6. 1. 7. 5. 8. 5. 7. 4. 7. 2. 7.
 5. 1. 6. 4. 7. 3. 3. 3. 5. 7. 6. 2. 5. 5. 3. 5. 2. 4. 3. 4. 2. 4. 3. 2.
 3. 4. 8. 6. 3. 7. 2. 5.]
Walls for the session are: [[4. 5.]
 [5. 4.]
 [5. 1.]
 [2. 1.]
 [7. 6.]
 [7. 6.]
 [4. 6.]
 [2. 3.]
 [2. 6.]
 [5. 4.]
 [

d:\tom\documents\swc\repos\octagon_analysis\trajectory_analysis\trajectory_vectors.py:321: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))


Time taken for infer_loser_choice_session (one session) is 0.71
Loser inferred choices: [1. 1. 1. 1. 7. 1. 4. 1. 6. 5. 3. 6. 1. 3. 6. 3. 8. 4. 1. 8. 4. 5. 5. 4.
 1. 3. 1. 8. 7. 6. 1. 1. 1. 1. 1. 8. 1. 3. 7. 8. 7. 6. 8. 4. 7. 3. 5. 7.
 5. 1. 1. 4. 3. 1. 4. 4. 2. 2. 8. 1. 2. 8. 1. 3. 1. 1. 5. 7. 4. 1. 3. 7.
 5. 3. 6. 1. 6. 3. 3. 7. 1. 7. 1. 2. 6. 1. 2. 1. 1. 5. 7. 1. 7. 1. 1. 1.
 1. 5. 2. 1. 1. 8. 2. 5.]
proportion loser's choice confident = 0.4230769230769231
Time taken for player_wall_choice_win_or_loss (one session for one player) is 0.79
Chosen walls are [4. 5. 1. 2. 7. 7. 4. 2. 2. 5. 7. 1. 7. 7. 6. 7. 1. 2. 4. 6. 8. 3. 1. 4.
 8. 2. 3. 4. 4. 3. 4. 8. 1. 1. 8. 5. 5. 4. 5. 5. 7. 7. 1. 4. 7. 1. 8. 6.
 7. 8. 7. 8. 2. 7. 4. 4. 2. 2. 8. 1. 2. 6. 1. 7. 5. 8. 5. 7. 4. 7. 2. 7.
 5. 1. 6. 4. 7. 3. 3. 3. 5. 7. 6. 2. 5. 5. 3. 5. 2. 4. 3. 4. 2. 4. 3. 2.
 3. 4. 8. 6. 3. 7. 2. 5.]
Walls for the session are: [[4. 5.]
 [5. 4.]
 [5. 1.]
 [2. 1.]
 [7. 6.]
 [7. 6.]
 [4. 6.]
 [2. 3.]
 [2. 6.]
 [5. 4.]
 [

...

Time taken for infer_loser_choice_session (one session) is 0.88
Loser inferred choices: [7. 6. 3. 3. 8. 7. 4. 1. 4. 1. 7. 2. 6. 2. 8. 5. 8. 1. 2. 1. 6. 4. 1. 8.
 7. 5. 4. 8. 5. 2. 7. 3. 8. 6. 5. 6. 3. 4. 6. 3. 2. 6. 6. 6. 1. 8. 5. 1.
 4. 4. 6. 4. 6. 2. 5. 2. 5. 6. 1. 2. 5. 3. 8. 4. 1. 7. 8. 1. 6. 1. 4. 3.
 5. 1. 7. 1. 4. 1. 7. 7. 1. 1. 7. 6. 5. 2. 2. 5. 7. 3. 6. 5. 3. 7. 8. 6.
 6. 3. 7. 1. 1. 4. 5. 1. 6. 4. 1. 8. 1. 8. 1. 1. 8. 2. 1. 2. 1. 3. 1. 1.
 1. 6. 6. 1. 1. 3. 6. 3. 2. 8. 4. 1. 8. 5. 2. 5. 4. 2. 3. 3. 6. 6. 2. 6.
 8. 2.]
proportion loser's choice confident = 0.4657534246575342
Time taken for player_wall_choice_win_or_loss (one session for one player) is 0.99
Chosen walls are [1. 8. 1. 5. 1. 7. 3. 6. 4. 5. 6. 3. 6. 3. 8. 7. 7. 7. 1. 1. 6. 8. 8. 4.
 7. 1. 5. 2. 6. 3. 2. 3. 4. 3. 5. 6. 4. 4. 6. 7. 5. 5. 6. 6. 5. 2. 4. 3.
 4. 7. 5. 2. 7. 1. 4. 2. 4. 5. 3. 2. 4. 3. 2. 6. 5. 6. 8. 1. 6. 8. 5. 3.
 2. 1. 1. 8. 3. 4. 4. 1. 2. 4. 3. 2. 1. 2. 2. 2. 4. 3. 4. 2. 4. 2. 1. 5.
 3. 3. 5. 3. 1. 1

d:\tom\documents\swc\repos\octagon_analysis\trajectory_analysis\trajectory_vectors.py:321: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))


Time taken for infer_loser_choice_session (one session) is 0.83
Loser inferred choices: [5. 1. 2. 8. 5. 1. 4. 1. 2. 7. 4. 4. 7. 3. 6. 8. 5. 6. 1. 6. 1. 8. 8. 1.
 5. 1. 7. 7. 5. 1. 1. 3. 3. 4. 1. 4. 1. 5. 4. 3. 6. 8. 4. 4. 6. 5. 8. 6.
 6. 1. 1. 3. 8. 2. 4. 1. 6. 7. 6. 1. 3. 1. 3. 8. 3. 6. 6. 5. 3. 4. 3. 7.
 2. 6. 2. 8. 4. 5. 8. 7. 8. 3. 8. 8. 1. 6. 3. 4. 4. 4. 7. 2. 1. 1. 7. 3.
 3. 1. 1. 8. 5. 6. 7. 3. 2. 4. 7. 1. 7. 7. 8. 4. 8. 4. 1. 1. 6. 2. 2. 5.
 1. 8. 1. 6. 5. 4. 4. 6. 3. 4. 8. 7. 5. 2. 8. 3. 8. 6. 1. 5. 2. 2. 6.]
proportion loser's choice confident = 0.6363636363636364
Time taken for player_wall_choice_win_or_loss (one session for one player) is 0.94
Chosen walls are [7. 8. 4. 8. 8. 4. 8. 2. 5. 8. 8. 1. 8. 3. 8. 7. 8. 6. 1. 6. 3. 6. 7. 2.
 3. 2. 1. 7. 8. 3. 8. 2. 2. 2. 8. 2. 1. 5. 7. 3. 5. 6. 3. 3. 5. 4. 6. 2.
 6. 1. 7. 7. 7. 8. 4. 1. 6. 7. 6. 1. 2. 6. 2. 6. 3. 1. 6. 5. 6. 5. 5. 6.
 7. 2. 6. 8. 4. 4. 2. 3. 6. 5. 4. 8. 1. 7. 7. 2. 2. 4. 2. 2. 2. 7. 8. 3.
 7. 5. 1. 6. 4. 5. 6. 4. 6.

d:\tom\documents\swc\repos\octagon_analysis\trajectory_analysis\trajectory_vectors.py:321: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
d:\tom\documents\swc\repos\octagon_analysis\analysis\loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))


Time taken for infer_loser_choice_session (one session) is 0.81
Loser inferred choices: [5. 1. 2. 8. 5. 1. 4. 1. 2. 7. 4. 4. 7. 3. 6. 8. 5. 6. 1. 6. 1. 8. 8. 1.
 5. 1. 7. 7. 5. 1. 1. 3. 3. 4. 1. 4. 1. 5. 4. 3. 6. 8. 4. 4. 6. 5. 8. 6.
 6. 1. 1. 3. 8. 2. 4. 1. 6. 7. 6. 1. 3. 1. 3. 8. 3. 6. 6. 5. 3. 4. 3. 7.
 2. 6. 2. 8. 4. 5. 8. 7. 8. 3. 8. 8. 1. 6. 3. 4. 4. 4. 7. 2. 1. 1. 7. 3.
 3. 1. 1. 8. 5. 6. 7. 3. 2. 4. 7. 1. 7. 7. 8. 4. 8. 4. 1. 1. 6. 2. 2. 5.
 1. 8. 1. 6. 5. 4. 4. 6. 3. 4. 8. 7. 5. 2. 8. 3. 8. 6. 1. 5. 2. 2. 6.]
proportion loser's choice confident = 0.6363636363636364
Time taken for player_wall_choice_win_or_loss (one session for one player) is 0.92
Chosen walls are [7. 8. 4. 8. 8. 4. 8. 2. 5. 8. 8. 1. 8. 3. 8. 7. 8. 6. 1. 6. 3. 6. 7. 2.
 3. 2. 1. 7. 8. 3. 8. 2. 2. 2. 8. 2. 1. 5. 7. 3. 5. 6. 3. 3. 5. 4. 6. 2.
 6. 1. 7. 7. 7. 8. 4. 1. 6. 7. 6. 1. 2. 6. 2. 6. 3. 1. 6. 5. 6. 5. 5. 6.
 7. 2. 6. 8. 4. 4. 2. 3. 6. 5. 4. 8. 1. 7. 7. 2. 2. 4. 2. 2. 2. 7. 8. 3.
 7. 5. 1. 6. 4. 5. 6. 4. 6.

In [ ]:
# run through the dictionary again, adding d2l, d2h, and first_visible for the opponent in social
for experiment_id, players in analysis_results.items():
    for player_id, data in players.items():
        
        trial_list_social = trial_lists_social[experiment_id]

        opponent_player_id = 1 if player_id == 0 else 0

        player_data = analysis_results[experiment_id][player_id]['social']
        opponent_data = analysis_results[experiment_id][opponent_player_id]['social']

        valid_indices_self = player_data['misc']['valid_trial_indices']
        valid_indices_opponent = opponent_data['misc']['valid_trial_indices']
        valid_indices_all = np.intersect1d(valid_indices_self, valid_indices_opponent)

        trial_list_social_filtered = trial_list_social
        player_data['regressors']['d2h_opponent'] = 




In [ ]:
import pickle

# with open('analysis_results.pkl', 'wb') as f:
#     pickle.dump(analysis_results, f)


# with open('analysis_results.pkl', 'rb') as f:
#     analysis_results = pickle.load(f)

#### Populate a dataframe, with a row for each trial, and fields for regressors (only including trials with fully-populated regressors)

In [51]:
glm_df_social = pd.DataFrame()

for session_id, players in analysis_results.items():
    for player_id in players:
        
        # take each filtered_regressor array and fill the relevant df field for this player
        player_data = analysis_results[session_id][player_id]['social']['regressors']
        choice = analysis_results[session_id][player_id]['social']['dependent']['choice']
        opponent_player_id = 1 if player_id == 0 else 1
        opponent_player_data = analysis_results[session_id][opponent_player_id]['social']['regressors']
        df_player = pd.DataFrame(
                    {
                        "SessionID" : session_id,
                        "PlayerID" : player_id,
                        "GlmPlayerID" : session_id*2 + player_id,
                        "ChooseHigh" : choice,
                        "WallSep" : player_data['wall_sep'],
                        "FirstSeenWall" : player_data['first_seen'],
                        "D2H" : player_data['d2h'],
                        "D2L" : player_data['d2l'],
                        "OpponentVisible" : player_data['opponent_visible'],
                        # "OpponentFirstSeenWall" : opponent_player_data['first_seen'],
                        # "OpponentD2H" : opponent_player_data['d2h'],
                        # "OpponentD2L" : opponent_player_data['d2l']
                        
                    }
        )


        # append this smaller dataframe to the the full dataframe
        glm_df_social = pd.concat([glm_df_social, df_player], ignore_index=True)



glm_df_social["FirstSeenWall"] = glm_df_social["FirstSeenWall"].astype("category")
glm_df_social["WallSep"] = glm_df_social["WallSep"].astype("category")

In [52]:
glm_df_solo = pd.DataFrame()

for session_id, players in analysis_results.items():
    for player_id in players:
        
        # take each filtered_regressor array and fill the relevant df field for this player
        player_data = analysis_results[session_id][player_id]['solo']['regressors']
        choice = analysis_results[session_id][player_id]['solo']['dependent']['choice']
        df_player = pd.DataFrame(
                    {
                        "SessionID" : session_id,
                        "PlayerID" : player_id,
                        "GlmPlayerID" : session_id,
                        "ChooseHigh" : choice,
                        "WallSep" : player_data['wall_sep'],
                        "FirstSeenWall" : player_data['first_seen'],
                        "D2H" : player_data['d2h'],
                        "D2L" : player_data['d2l']
                    }
        )

        # append this smaller dataframe to the the full dataframe
        glm_df_solo = pd.concat([glm_df_solo, df_player], ignore_index=True)


glm_df_solo["FirstSeenWall"] = glm_df_solo["FirstSeenWall"].astype("category")
glm_df_solo["WallSep"] = glm_df_solo["WallSep"].astype("category")

In [59]:
glm_df_social['FirstSeenWall'][400:450]

400    2.0
401    2.0
402    2.0
403    2.0
404    1.0
405    2.0
406    2.0
407    1.0
408    2.0
409    1.0
410    1.0
411    1.0
412    2.0
413    1.0
414    1.0
415    1.0
416    1.0
417    1.0
418    1.0
419    1.0
420    1.0
421    2.0
422    1.0
423    1.0
424    2.0
425    2.0
426    2.0
427    2.0
428    2.0
429    1.0
430    2.0
431    1.0
432    1.0
433    2.0
434    1.0
435    1.0
436    2.0
437    2.0
438    2.0
439    2.0
440    1.0
441    2.0
442    1.0
443    2.0
444    1.0
445    1.0
446    2.0
447    1.0
448    1.0
449    2.0
Name: FirstSeenWall, dtype: category
Categories (2, float64): [1.0, 2.0]

#### GLM Code in Pymer4

In [55]:
!R --version


R version 4.1.3 (2022-03-10) -- "One Push-Up"
Copyright (C) 2022 The R Foundation for Statistical Computing
Platform: x86_64-w64-mingw32/x64 (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under the terms of the
GNU General Public License versions 2 or 3.
For more information about these matters see
https://www.gnu.org/licenses/.



In [56]:
from pymer4.models import Lmer
import pandas as pd

df = pd.DataFrame({
    "y": [0, 1, 0, 1, 1, 0],
    "x": [1, 2, 3, 4, 5, 6],
    "group": ["A", "A", "B", "B", "C", "C"]
})

model = Lmer("y ~ x + (1|group)", data=df, family='binomial')
model.fit()

boundary (singular) fit: see help('isSingular') 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: y~x+(1|group)

Family: binomial	 Inference: parametric

Number of observations: 6	 Groups: {'group': 3.0}

Log-likelihood: -4.130 	 AIC: 14.260

Random effects:

              Name  Var  Std
group  (Intercept)  0.0  0.0

No random effect correlations specified

Fixed effects:



d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),-0.402,-4.079,3.275,1.876,0.669,0.017,26.442,0.401,0.017,0.964,-0.214,0.830,
x,0.115,-0.830,1.060,0.482,1.122,0.436,2.886,0.529,0.304,0.743,0.238,0.812,


In [57]:
glm_df_solo['ChooseHigh']

0       1.0
1       0.0
2       1.0
3       1.0
4       0.0
       ... 
2791    0.0
2792    1.0
2793    1.0
2794    1.0
2795    1.0
Name: ChooseHigh, Length: 2796, dtype: float64

In [ ]:
from pymer4.models import Lmer

model_formula = 'ChooseHigh ~ D2L + D2H + FirstSeenWall + WallSep + (1|GlmPlayerID)'
# model_formula = 'ChooseHigh ~ D2H + D2L + FirstSeenWall + WallSep + (D2H + D2L + FirstSeenWall + WallSep | GlmPlayerID)'
# model_formula = 'ChooseHigh ~ D2H + D2L + FirstSeenWall + WallSep + (1 + D2H + D2L + FirstSeenWall || GlmPlayerID)'


model = Lmer(model_formula, data=glm_df_solo, family='binomial')
results=model.fit()

print(results)

d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\rpy2\robjects\pandas2ri.py:65: UserWarning: Error while trying to convert the column "WallSep". Fall back to string conversion. The error is: Converting pandas "Category" series to R factor is only possible when categories are strings.
  warnings.warn('Error while trying to convert '
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\rpy2\robjects\pandas2ri.py:65: UserWarning: Error while trying to convert the column "FirstSeenWall". Fall back to string conversion. The error is: Converting pandas "Category" series to R factor is only possible when categories are strings.
  warnings.warn('Error while trying to convert '


[1] "failure to converge in 10000 evaluations"
[2] " \n"                                     

Model failed to converge with max|grad| = 0.74372 (tol = 0.002, component 1) 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: ChooseHigh~D2H+D2L+FirstSeenWall+WallSep+(D2H+D2L+FirstSeenWall+WallSep|GlmPlayerID)

Family: binomial	 Inference: parametric

Number of observations: 2796	 Groups: {'GlmPlayerID': 32.0}

Log-likelihood: -888.690 	 AIC: 1831.381

Random effects:

                         Name    Var    Std
GlmPlayerID       (Intercept)  1.545  1.243
GlmPlayerID               D2H  0.003  0.056
GlmPlayerID               D2L  0.006  0.075
GlmPlayerID  FirstSeenWall2.0  1.650  1.285
GlmPlayerID        WallSep2.0  0.328  0.573
GlmPlayerID        WallSep4.0  0.851  0.922

                          IV1               IV2   Corr
GlmPlayerID       (Intercept)               D2H -0.358
GlmPlayerID       (Intercept)               D2L -0.189
GlmPlayerID       (Intercept)  FirstSe

d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:744: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_corrs = ran_corrs.applymap(


In [58]:
print(results)

                  Estimate  2.5_ci  97.5_ci     SE     OR  OR_2.5_ci  OR_97.5_ci   Prob  Prob_2.5_ci  Prob_97.5_ci  Z-stat  P-val Sig
(Intercept)         -0.112  -0.458    0.234  0.176  0.894      0.633       1.263  0.472        0.388         0.558  -0.635  0.525    
D2H                  0.011  -0.001    0.023  0.006  1.011      0.999       1.023  0.503        0.500         0.506   1.859  0.063   .
D2L                 -0.005  -0.016    0.007  0.006  0.995      0.984       1.007  0.499        0.496         0.502  -0.787  0.431    
FirstSeenWall2.0    -0.029  -0.184    0.125  0.079  0.971      0.832       1.133  0.493        0.454         0.531  -0.371  0.711    
WallSep2.0           0.036  -0.148    0.221  0.094  1.037      0.862       1.247  0.509        0.463         0.555   0.385  0.700    
WallSep4.0           0.031  -0.149    0.212  0.092  1.032      0.862       1.236  0.508        0.463         0.553   0.342  0.733    


In [84]:
from pymer4.models import Lmer

# model_formula = 'ChooseHigh ~ D2L + D2H + FirstSeenWall + WallSep + OpponentVisible + (1|GlmPlayerID)'
# model_formula = 'ChooseHigh ~ D2H + D2L + FirstSeenWall + WallSep + OpponentVisible + (D2H + D2L + FirstSeenWall + WallSep | GlmPlayerID)'
model_formula = 'ChooseHigh ~ D2H + D2L + FirstSeenWall + WallSep + OpponentVisible + (1 + D2H + D2L + FirstSeenWall + OpponentVisible || GlmPlayerID)'


model = Lmer(model_formula, data=glm_df_social, family='binomial')
results=model.fit()

print(results)

d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\rpy2\robjects\pandas2ri.py:65: UserWarning: Error while trying to convert the column "WallSep". Fall back to string conversion. The error is: Converting pandas "Category" series to R factor is only possible when categories are strings.
  warnings.warn('Error while trying to convert '
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\rpy2\robjects\pandas2ri.py:65: UserWarning: Error while trying to convert the column "FirstSeenWall". Fall back to string conversion. The error is: Converting pandas "Category" series to R factor is only possible when categories are strings.
  warnings.warn('Error while trying to convert '


Model failed to converge with max|grad| = 0.0875188 (tol = 0.002, component 1) 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: ChooseHigh~D2H+D2L+FirstSeenWall+WallSep+OpponentVisible+(1+D2H+D2L+FirstSeenWall+OpponentVisible||GlmPlayerID)

Family: binomial	 Inference: parametric

Number of observations: 3521	 Groups: {'GlmPlayerID': 64.0}

Log-likelihood: -1568.587 	 AIC: 3165.174

Random effects:

                           Name    Var    Std
GlmPlayerID         (Intercept)  0.000  0.009
GlmPlayerID.1               D2H  0.001  0.031
GlmPlayerID.2               D2L  0.000  0.022
GlmPlayerID.3  FirstSeenWall1.0  0.192  0.438
GlmPlayerID.3  FirstSeenWall2.0  0.575  0.758
GlmPlayerID.4   OpponentVisible  0.196  0.443

                            IV1               IV2   Corr
GlmPlayerID.3  FirstSeenWall1.0  FirstSeenWall2.0 -0.999

Fixed effects:

                  Estimate  2.5_ci  97.5_ci     SE      OR  OR_2.5_ci  OR_97.5_ci   Prob  Prob_2.5_ci  Prob_97.5_ci  Z-stat

d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:744: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_corrs = ran_corrs.applymap(


In [52]:
print(results)

                  Estimate  2.5_ci  97.5_ci     SE      OR  OR_2.5_ci  OR_97.5_ci   Prob  Prob_2.5_ci  Prob_97.5_ci  Z-stat  P-val  Sig
(Intercept)          3.204   2.774    3.634  0.219  24.638     16.025      37.879  0.961        0.941         0.974  14.601  0.000  ***
D2H                 -0.081  -0.094   -0.067  0.007   0.922      0.910       0.935  0.480        0.476         0.483 -11.807  0.000  ***
D2L                 -0.072  -0.086   -0.058  0.007   0.931      0.918       0.944  0.482        0.479         0.486 -10.011  0.000  ***
FirstSeenWall2.0    -0.186  -0.328   -0.044  0.072   0.830      0.721       0.957  0.454        0.419         0.489  -2.567  0.010    *
WallSep2.0           0.125  -0.047    0.298  0.088   1.134      0.954       1.347  0.531        0.488         0.574   1.423  0.155     
WallSep4.0           0.145  -0.022    0.311  0.085   1.156      0.978       1.365  0.536        0.495         0.577   1.703  0.088    .
